In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Giải nén dữ liệu

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

import sklearn
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

import scipy
from scipy.sparse import hstack
import gc


# Đọc dữ liệu

In [ ]:
train = pd.read_csv('train.tsv', sep = '\t')
test = pd.read_csv('test_stg2.tsv', sep = '\t')

# Trực quan hóa và phân tích dữ liệu

Sau khi đã load dữ liệu từ file train và test, tiến hành trực quan hóa và phân tích dữ liệu. </br>
Đầu tiên ta sẽ xem xét một vài bản ghi trong file train và test để quan sát các trường dữ liệu cũng như dữ liệu chứa trong đó.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print('Train\'s shape: ', train.shape)
print('Test\'s shape: ', test.shape)

Nhận xét: tập dữ liệu huấn luyện gồm 1482535 bản ghi với 8 thuộc tính, tập test gồm 693359 bản ghi với 7 thuộc tính (không có trường dữ liệu price mà cần phải dự đoán). Ta sẽ tiến hành phân tích, xử lý dữ liệu trên cả 2 tập train và test, sau đó huấn luyện mô hình dựa trên tập huấn luyện

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum()/train.shape[0]

- Thông qua thống kê ban đầu, có thể thấy:
    - Có 4 thuộc tính dạng số và 4 thuộc tính dạng object, tuy nhiên, có thể thấy item_condition_id và shipping đều là 2 thuộc tính phân loại (category), do đó ta sẽ chuyển đổi kiểu dữ liệu của các thuộc tính này từ dạng số sang category
    - Có 3 trường dữ liệu bị khuyết giá trị, hay nói cách khác là chứa giá trị null. Qua thống kê số bản ghi chứa giá trị null, ta có 3 thuộc tính là category_name (chứa 6327 bản ghi null), brand_name (chứa 632682 bản ghi null) và item_description (chứa 4 bản ghi null).
    - Số bản ghi chứa giá trị null chiếm đến gần 43% dữ liệu → không thể xóa hết các bản ghi này. Ta sẽ điền vào các giá trị còn thiếu là “missing value”

In [ ]:
train.describe(include="all")

Trên đây là thống kê tóm tắt về các trường dữ liệu, trong đó:
- Với dữ liệu dạng số, thu được các con số thống kê về số lượng bản ghi, trung bình, độ lệch chuẩn, giá trị nhỏ nhất, 25%, 50%, 75% và giá trị lớn nhất
- Với dữ liệu object, thu được số bản ghi, tổng các giá trị unique (khác biệt), tần suất xuất hiện nhiều nhất của dữ liệu ứng với từng thuộc tính </br>
Chi tiết
- Phần lớn name là unique
- 75% item_condition_id là <= 3 -> các giá trị 4, 5 khá ít
- Danh mục Women, clothing xuất hiện nhiều nhất
- PINK là thương hiệu được rao bán nhiều nhất
- Có sự thiên lệch trong phân phối price
- Thuộc tính shipping có phân phối khá đều giữa 2 giá trị 1 và 0
- Phần lớn mặt hàng không có mô tả (No description yet)

# # Phân tích cụ thể từng thuộc tính

# # # 1. price

In [ ]:
train['price'].describe()

Nhận xét: giá trị trung bình của price là 26.7, 75% dữ liệu là 29, tuy nhiên giá trị lớn nhất lại là 2009 (đơn vị USD), lớn hơn rất nhiều so với trung bình tổng thế -> đây có thể là giá trị ngoại lai. Ta sẽ trực quan hóa để làm rõ nhận định

In [ ]:
sns.histplot(train['price'], bins=50)

In [ ]:
len(train[train['price']>400])

Qua biểu đồ mật độ phân bố (hisplot) về price, có thể thấy các giá trị từ 400 trở lên không được thể hiện rõ, mặt khác, biểu đồ còn có dạng phân phối lệch về bên trái -> cần đưa về dạng phân phối chuẩn, vì với phân phối chuẩn dữ liệu sẽ được phân phối đồng đều hơn, mặt khác đây lại là điểm cộng cho mô hình hồi quy vì giả thuyết đầu vào mô hình thường theo phân phối chuẩn. Bên cạnh đó số bản ghi có price>400 cũng không quá nhiều (1917) nên có thể xem xét để xóa. Tuy nhiên với dữ liệu có phân bố như trên, khi biểu diễn bằng giá trị log của dữ liệu sẽ thu được dạng phân phối chuẩn mong muốn. Do đó hướng giải quyết sẽ là biểu diễn log thay vì loại bỏ dữ liệu

In [ ]:
# Biểu đồ biểu diễn log(price + 1), cộng thêm 1 để tránh các bản ghi có price=0 dẫn đến giá trị vô cùng
sns.kdeplot(train['price'].apply(lambda x:np.log(x+1)))

Bây giờ mục tiêu của ta sẽ là dự đoán log(price+1) thay vì price như ban đầu

In [ ]:
train['price'] = np.log1p(train['price'])

In [ ]:
train.head()

# # # 2. item_condition_id

In [ ]:
pd.value_counts(train['item_condition_id'])

In [ ]:
train['item_condition_id'].value_counts(sort=True).plot.bar()

item_condition_id là thuộc tính phân loại có thứ tự (category-ordinal), gồm 5 giá trị từ 1-5 (1 biểu thị sản phẩm có tình trạng tốt và giảm dần về 5-tình trạng xấu) . Qua thống kê có thể thấy giá trị 1 chiếm số lượng nhiều nhất và 5 thấp nhất, thấp hơn rất nhiều so với các giá trị khác. Điều này khá dễ hiểu vì không người bán nào muốn đánh giá chất lượng sản phẩm của mình ở mức thấp cả 

In [ ]:
# Biểu đồ phân bố log_price theo item_condition_id
sns.displot(data=train, x='price', hue='item_condition_id', kind="kde", fill=True)

Nhận xét : những mặt hàng có tình trạng kém (=5) có mật độ price khá thấp. Điều này khá dễ hiểu vì mặt hàng có điều kiện xấu tất nhiên giá sẽ thấp

# # # 3. shipping

In [ ]:
pd.value_counts(train['shipping'])

In [ ]:
train['shipping'].value_counts().plot.pie()

Shipping gồm 2 giá trị là 0 và 1 có số lượng bản ghi tương ứng không quá chênh lệch

In [ ]:
sns.boxplot(data=train, y='price', x='shipping')

Nhận xét : price cho shipping=1 thấp hơn so với shipping=0, có thể nhận định rằng nếu người bán trả phí ship thì bù lại sản phẩm sẽ rẻ hơn và ngược lại

# # # 4. brand_name

In [ ]:
counts = train['brand_name'].value_counts()
percentages = train['brand_name'].value_counts(normalize=True)
pd.DataFrame({'counts': counts, 'per':percentages})

In [ ]:
brand_order = train['brand_name'].value_counts().head(10).index
plt.figure(figsize=(9, 5))
sns.countplot(data=train, y='brand_name', order=brand_order)
plt.title('Top 10 brand used')
plt.xlabel('Counts')
plt.ylabel('brand_name')

Nhận xét : top 10 thương hiệu được rao bán nhiều nhất là PINK, Nike, Victoria's Secret, LuLaRoe, Apple, FOREVER 21, Nintendo, Lululemon, Michael Kors, American Eagle, trong đó PINK là thương hiệu chiếm nhiều tỉ lệ bản ghi nhất 

# # # 5. category_name

In [ ]:
train['category_name'].value_counts()

In [ ]:
category_order = train['category_name'].value_counts().head(10).index
plt.figure(figsize=(9, 5))
sns.countplot(data=train, y='category_name', order=category_order)
plt.title('Top 10 category used')
plt.xlabel('Counts')
plt.ylabel('category_name')

Có 1287 tên danh mục riêng biệt, trong đó tỉ lệ danh mục Women, Tights, Leggings chiếm nhiều nhất. Tên danh mục tại Mercari cũng được phân cấp và ngăn cách các cấp bởi dấu '/'

# # # 6. name

In [ ]:
train['name'].describe()

# # # 7. item_description

In [ ]:
counts = train['item_description'].value_counts()
percentages = train['item_description'].value_counts(normalize=True)
pd.DataFrame({'counts': counts, 'per':percentages})

In [ ]:
train.head()

# Tiền xử lý dữ liệu

In [ ]:
# Phân chia category_name thành 3 cấp tương ứng

def category_split(category_name):
    try:
        return category_name.split('/')
    except:
        return ['missing value', 'missing value', 'missing value']
    
train['cate_lv1'], train['cate_lv2'], train['cate_lv3'] = zip(*train['category_name'].apply(lambda x: category_split(x)))
test['cate_lv1'], test['cate_lv2'], test['cate_lv3'] = zip(*test['category_name'].apply(lambda x: category_split(x)))

def data_preprocess(data):
    
    # Xử lý các giá trị null : category_name, brand_name, item_description
    
    data['category_name'].fillna('missing value', inplace=True)
    data['brand_name'].fillna('missing value', inplace=True)
    data['item_description'].fillna('missing value', inplace=True)
    
    # Chuyển đổi kiểu dữ liệu của các thuộc tính category_name, brand_name, item_condition_id, shipping về dạng category

    data['cate_lv1'] = data['cate_lv1'].astype("category")
    data['cate_lv2'] = data['cate_lv2'].astype("category")
    data['cate_lv3'] = data['cate_lv3'].astype("category")
    
    data['brand_name'] = data['brand_name'].astype("category")
    data['item_condition_id'] = data['item_condition_id'].astype("category")
    data['shipping'] = data['shipping'].astype("category")

   
data_preprocess(train)
data_preprocess(test)

In [ ]:
train.head()

# Trích xuất đặc trưng

- Với đầu vào dạng text, sử dụng mã hóa bag of word và tf-idf
    - Áp dụng bag of word cho thuộc tính name vì chúng khá ngắn
        
        **Bag of word :**
        
        - Là kĩ thuật trích xuất đặc trưng từ văn bản bằng cách đếm số lần xuất hiện của các token, với mỗi token sẽ có 1 feature column, được gọi là **text vectorization.**
    - Áp dụng tf-idf cho item_description vì dữ liệu mô tả chứa rất nhiều từ
        
        **Tf-idf :**
        
        - Tương tự như BoW nhưng thay vì đếm số lần xuất hiện của token, ta sẽ sử dụng giá trị tf-idf, trong đó :
            
            **tf (term frequency)** - tần suất từ
            
            $$
            \mathrm{tf}\{t, d\} = \begin{cases}1+\log{10} count(t, d)&\textrm{nếu } count(t, d) > 0\\ 0 &\textrm{ngược lại}\end{cases}
            $$
            
            với $count(t,d)$ là số lần xuất hiện của từ $t$ trong văn bản (ngữ cảnh lân cận) $d$.
            
            **idf (Inverse document frequency) - nghịch đảo tần suất văn bản**
            
            $$
            \mathrm{idf}_t = \log\frac N {\mathrm{df}_t}
            $$
            
            với $N$ là số lượng văn bản, $\mathrm{df}_t$ là số văn bản có từ $t$.
            
            **Giá trị tf-idf**
            
            $$
            w_{t,d} = \mathrm{tf}_\{t, d\} \times \mathrm{idf}_t
            $$
            
- Với đầu vào dạng category, sử dụng mã hóa LabelBinarizer
    
    **LabelBinarizer:** 
    
    Tương tự mã hóa one-hot encoding, tuy nhiên, OneHotEncoder cần dữ liệu ở dạng mã hóa số nguyên trước tiên để chuyển đổi thành mã hóa tương ứng, điều này không bắt buộc trong trường hợp của LabelBinarizer.

In [ ]:
cnt_vec = CountVectorizer()

X_train_name = cnt_vec.fit_transform(train['name'])
X_test_name = cnt_vec.transform(test['name'])

In [ ]:
print(X_train_name.shape)
print(X_test_name.shape)

In [ ]:
tfidf_descp = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), stop_words='english')

X_train_descp = tfidf_descp.fit_transform(train['item_description'])
X_test_descp = tfidf_descp.transform(test['item_description'])

In [ ]:
print('brand_name processing...')
lb_brand_name = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand_name.fit_transform(train['brand_name'])
X_test_brand = lb_brand_name.transform(test['brand_name'])

print('item_condition_id processing...')
lb_item_cond_id = LabelBinarizer(sparse_output=True)
X_train_item_condition_id = lb_item_cond_id.fit_transform(train['item_condition_id'])
X_test_item_condition_id = lb_item_cond_id.transform(test['item_condition_id'])

print('shipping processing...')
lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train['shipping'])
X_test_shipping = lb_shipping.transform(test['shipping'])

print('cate_lv1 processing...')
lb_main_cat = LabelBinarizer(sparse_output=True)
X_train_main_cat = lb_main_cat.fit_transform(train['cate_lv1'])
X_test_main_cat = lb_main_cat.transform(test['cate_lv1'])

print('cate_lv2 processing...')
lb_sub_cat = LabelBinarizer(sparse_output=True)
X_train_sub_cat = lb_sub_cat.fit_transform(train['cate_lv2'])
X_test_sub_cat = lb_sub_cat.transform(test['cate_lv2'])

print('cate_lv3 processing...')
lb_item_cat = LabelBinarizer(sparse_output=True)
X_train_item_cat = lb_item_cat.fit_transform(train['cate_lv3'])
X_test_item_cat = lb_item_cat.transform(test['cate_lv3'])

In [ ]:
from scipy.sparse import hstack
import gc

sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

X_train = hstack(sparse_matrix_list).tocsr()
print(type(X_train), X_train.shape)

del X_train
gc.collect()

# Evaluation

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluate_orig_price(y_test, preds):
    preds_exmpm = np.expm1(preds)
    y_test_exmpm = np.expm1(y_test)
    
    return rmsle(y_test_exmpm, preds_exmpm)

def model_train_predict(model, matrix_list):
    X = hstack(matrix_list).tocsr()
    X_train, X_test, y_train, y_test = train_test_split(X, train['price'], test_size=0.2)

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    del X, X_train, X_test, y_train
    gc.collect()
    
    return preds, y_test

# Mô hình

RidgeRegression model

In [ ]:
linear_model = Ridge(solver='lsqr', fit_intercept=False)

sparse_matrix_list = (X_train_name, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

linear_preds, y_test = model_train_predict(model=linear_model, 
                                           matrix_list=sparse_matrix_list)

print('Item Description rmsle:', evaluate_orig_price(y_test, linear_preds))

sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)


linear_preds, y_test = model_train_predict(model=linear_model, 
                                           matrix_list=sparse_matrix_list)
print('Item Description rmsle:', evaluate_orig_price(y_test, linear_preds))

In [ ]:
sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

X_train = hstack(sparse_matrix_list).tocsr()

sparse_matrix_list = (X_test_name, X_test_descp, X_test_brand, 
                      X_test_item_condition_id, X_test_shipping, 
                      X_test_main_cat, X_test_sub_cat, X_test_item_cat)
X_test = hstack(sparse_matrix_list).tocsr()

y_train = train['price']
linear_model.fit(X_train, y_train)
preds = linear_model.predict(X_test)
preds = np.expm1(preds)


In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission.loc[:, 'price'] = preds
submission.to_csv('submission.csv', index=False)